In [169]:
## ADVENT OF CODE 2022, Day 18
## Edmund Dickinson, Python implementation

# File read
input_folder = "Input/"
input_file = "day18a.txt"
file_path = input_folder + input_file

with open(file_path) as file:
    input = file.read().splitlines()

In [170]:
droplet_lst = []
for line in input:
    droplet_lst.append(tuple([int(i) for i in line.split(",")]))

droplet = set(droplet_lst)

In [171]:
def connectedFaces(block,droplet):
    # block - coordinates of block
    # droplet - list of coordinates of blocks already present
    xb,yb,zb = block
    connected = 0

    for sub in droplet:
        # Compare to each block already present
        # Shares a face if aligned with the block in a cardinal direction
        xs,ys,zs = sub

        if(
            (abs(xs-xb) == 1 and ys == yb and zs == zb)
            or
            (abs(ys-yb) == 1 and xs == xb and zs == zb)
            or
            (abs(zs-zb) == 1 and xs == xb and ys == yb)
        ):
            connected += 1

    return connected

def getSurfaceArea(blocks):
    droplet = []
    num_faces = 0

    # Note(ED): Does sorting guarantee connectivity..?
    for block in sorted(blocks):
        num_faces += 6 - 2*connectedFaces(block,droplet)
        droplet.append(block)       

    return num_faces

def getNeighbours(coord, droplet):
    # Return cardinal neighbours of coord that are not in droplet
    x,y,z = coord
    neighbours = set([
        (x+1,y,z),
        (x-1,y,z),
        (x,y+1,z),
        (x,y-1,z),
        (x,y,z+1),
        (x,y,z-1)
    ])

    return neighbours.difference(droplet)

In [172]:
# Determine total surface area
surface_area_tot = getSurfaceArea(droplet)
print( surface_area_tot )

3586


In [173]:
# Determine external surface area only

# Determine the cube of space surrounding the droplet
# Note(ED): could presumably be done with min() max() functions?
xmin = 0
xmax = 0
ymin = 0
ymax = 0
zmin = 0
zmax = 0

for block in droplet:
    x,y,z = block

    if (x < xmin):
        xmin = x
    if (x > xmax):
        xmax = x
    if (y < ymin):
        ymin = y
    if (y > ymax):
        ymax = y
    if (z < zmin):
        zmin = z
    if (z > zmax):
        zmax = z

# Search zone is all blocks surrounding the 3D bounds of the droplet
zone = set()
distance = {}
for x in range(xmin-1,xmax+2):
    for y in range(ymin-1,ymax+2):
        for z in range(zmin-1,zmax+2):
            coord = x,y,z
            
            zone.add(coord)
            distance[coord] = -1

# List of coordinates of external blocks
block_extern = set()

# Start from a definitely external location
loc_init = (xmin-1,ymin-1,zmin-1)
block_extern.add(loc_init)
distance[loc_init] = 0
currDistance = 0

# Propagate outwards through connected external blocks, until opposite end of zone is reached
while ( not((xmax+1,ymax+1,zmax+1) in block_extern) ):
    source = [coord for coord in zone if distance[coord] == currDistance]
    currDistance += 1

    for coord in source:
        for dest in getNeighbours(coord, droplet):
            if ((dest in zone) and (distance[dest] == -1)):
                # Not yet visited
                distance[dest] = currDistance
                block_extern.add(dest)

# Evaluate the filled droplet as the complement of the external blocks
droplet_filled = zone.difference(block_extern)

# Evaluate the interior void as (zone - external - droplet)
# Surface area calculation assumes that the void is connected - seems to work
void = zone.difference(block_extern).difference(droplet)
surface_area_ext = surface_area_tot - getSurfaceArea(void)

# External surface area is surface area of the filled droplet
print ( surface_area_ext )

2072
